# Doom Game Using PPO

## Initialize VizDoom

In [1]:
# import VizDoom for game env
from vizdoom import *
# Import random for action sampling
import random
# Import time for sleeping
import time
# import numpy for identity matrix
import numpy as np

from matplotlib import pyplot as plt

## Make it a Gym Env

In [2]:
# Import environment base class from OpenAI Gym
from gymnasium import Env
# Import gym spaces
from gymnasium.spaces import Discrete, Box
# Import Opencv for greyscaling observations
import cv2

In [3]:
# Create VizDoom OpenAI Gym Environment
class VizDoomGym(Env): 
    def __init__(self, render=False):
        """
        Function called when we start the env.
        """

        # Inherit from Env
        super().__init__()
        
        # Set up game
        self.game = DoomGame()
        self.game.load_config('VizDoom/scenarios/basic.cfg')
        self.game.set_window_visible(render)
        self.game.init()

        # Whether we want to render the game 
        if render == False:
            self.game.set_window_visible(False)
        else:
            self.game.set_window_visible(True)

        # Start the game
        self.game.init()
        
        # Create action space and observation space
        self.observation_space = Box(low=0, high=255, shape=(100, 160, 1), dtype=np.uint8)
        self.action_space = Discrete(3)

    
    def step(self, action):
        """
        How we take a step in the environment.
        """

        # Specify action and take step
        actions = np.identity(3, dtype=np.uint8)
        reward = self.game.make_action(actions[action], 4) # get action using index -> left, right, shoot
        
        # Get all the other stuff we need to return 
        if self.game.get_state():  # if nothing is
            state = self.game.get_state().screen_buffer
            state = self.grayscale(state)  # Apply Grayscale
            ammo = self.game.get_state().game_variables[0] 
            info = ammo
        # If we dont have anything turned from game.get_state
        else:
            # Return a numpy zero array
            state = np.zeros(self.observation_space.shape)
            # Return info (game variables) as zero
            info = 0

        info = {"info":info}
        done = self.game.is_episode_finished()
        truncated = False  # Assuming it's not truncated, modify if applicable
        
        return state, reward, done, truncated, info

    
    def render(self):
        """
        Define how to render the game environment.
        """
        pass

    
    def reset(self, seed=None):
        """
        Function for defining what happens when we start a new game.
        """
        # Set seed if provided
        if seed is not None:
            self.game.set_seed(seed)
    
        self.game.new_episode()  # Start a new episode regardless of the seed

        # After starting a new episode, get the initial state
        if self.game.get_state() is not None:
            state = self.game.get_state().screen_buffer
            return self.grayscale(state)
        else:
            # Return a zero array if no state is available (e.g., at episode start)
            return np.zeros(self.observation_space.shape, dtype=np.uint8)


    
    def grayscale(self, observation):
        """
        Function to grayscale the game frame and resize it.
        observation: gameframe
        """
        # Change colour channels 
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)

        # Reduce image pixel size for faster training
        resize = cv2.resize(gray, (160,100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize,(100, 160,1))
        return state

    def close(self):
        """
        Call to close down the game.
        """
        self.game.close()


## Custom Pytorch PPO Model

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

### Define the Actor-Critic Model

In [6]:
def conv2d_size_out(size, kernel_size = 3, stride = 2, padding = 0):
    return (size + 2 * padding - (kernel_size - 1) - 1) // stride  + 1

# Calculate output size after each convolutional layer
h_w = [100, 160]  # initial height and width
h_w = [conv2d_size_out(x, 8, 4) for x in h_w]  # after first conv layer
h_w = [conv2d_size_out(x, 4, 2) for x in h_w]  # after second conv layer
h_w = [conv2d_size_out(x, 3, 1) for x in h_w]  # after third conv layer

# Calculate the total number of features before the linear layer
feature_count = 64 * h_w[0] * h_w[1]

class ActorCriticNetwork(nn.Module):
    def __init__(self, in_channels, n_output):
        super(ActorCriticNetwork, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        
        # Temporarily assume some output size after convolution
        # This should ideally be calculated based on input size
        self.feature_count = 64 * conv2d_size_out(conv2d_size_out(conv2d_size_out(100, 8, 4), 4, 2), 3, 1) * \
                        conv2d_size_out(conv2d_size_out(conv2d_size_out(160, 8, 4), 4, 2), 3, 1)

        self.fc = nn.Linear(self.feature_count, 512)
        self.actor = nn.Linear(512, n_output)
        self.critic = nn.Linear(512, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.reshape(-1, self.feature_count)  # Use reshape instead of view
        x = F.relu(self.fc(x))
        action_probs = F.softmax(self.actor(x), dim=1)
        value = self.critic(x)
        return action_probs, value

### Implement Helper Functions

In [7]:
def compute_returns(next_value, rewards, masks, gamma=0.99):
    """
    Calculate the returns using the rewards and the next state's value estimate.
    """
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def ppo_update(policy_net, optimizer, states, actions, log_probs_old, returns, advantages, clip_param=0.2):
    """
    Perform one update step of the PPO algorithm.
    """
    for _ in range(10):  # PPO epochs
        # Get the log probabilities and state values from the model
        log_probs, state_values = policy_net(states)
        # Select the log probabilities for the actions taken
        log_probs = log_probs.gather(1, actions.unsqueeze(-1)).squeeze(-1)
        # Calculate entropy to encourage exploration
        entropy = -(log_probs * torch.exp(log_probs)).sum(1, keepdim=True).mean()

        # Calculate the ratio (pi_theta / pi_theta_old) for the actions taken
        ratios = torch.exp(log_probs - log_probs_old)
        surr1 = ratios * advantages
        surr2 = torch.clamp(ratios, 1.0 - clip_param, 1.0 + clip_param) * advantages
        policy_loss = -torch.min(surr1, surr2).mean()
        value_loss = F.smooth_l1_loss(state_values.squeeze(-1), returns)
        loss = policy_loss + 0.5 * value_loss - 0.01 * entropy

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


### Training Loop

In [8]:
def train(env, model, num_episodes, device):
    optimizer = optim.Adam(model.parameters(), lr=0.0003)
    model.to(device)
    gamma = 0.99

    for episode in range(num_episodes):
        state = env.reset()
        state = torch.from_numpy(state).float().unsqueeze(0).permute(0, 3, 1, 2).to(device)
        done = False
        total_reward = 0

        while not done:
            policy_dist, value = model(state)
            action = policy_dist.multinomial(num_samples=1).detach()
            next_state, reward, done, _, _ = env.step(action.item())
            next_state = torch.from_numpy(next_state).float().unsqueeze(0).permute(0, 3, 1, 2).to(device)

            total_reward += reward

            # Calculate and update loss here

            state = next_state

        print(f"Episode {episode + 1}: Completed. Total Reward: {total_reward}")

if __name__ == "__main__":
    env = VizDoomGym(render=True)
    in_channels = 1  # Assuming grayscale input
    n_actions = env.action_space.n
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ActorCriticNetwork(in_channels, n_actions)
    num_episodes = 500
    train(env, model, num_episodes, device)

Episode 1: Completed. Total Reward: -380.0
Episode 2: Completed. Total Reward: 5.0
Episode 3: Completed. Total Reward: -390.0
Episode 4: Completed. Total Reward: 95.0
Episode 5: Completed. Total Reward: -385.0
Episode 6: Completed. Total Reward: 95.0
Episode 7: Completed. Total Reward: -380.0
Episode 8: Completed. Total Reward: -380.0
Episode 9: Completed. Total Reward: 95.0
Episode 10: Completed. Total Reward: -380.0
Episode 11: Completed. Total Reward: 95.0
Episode 12: Completed. Total Reward: -385.0
Episode 13: Completed. Total Reward: -385.0
Episode 14: Completed. Total Reward: -390.0
Episode 15: Completed. Total Reward: 95.0
Episode 16: Completed. Total Reward: -390.0
Episode 17: Completed. Total Reward: 95.0
Episode 18: Completed. Total Reward: -390.0
Episode 19: Completed. Total Reward: -390.0
Episode 20: Completed. Total Reward: -10.0
Episode 21: Completed. Total Reward: 95.0
Episode 22: Completed. Total Reward: -385.0
Episode 23: Completed. Total Reward: -385.0
Episode 24: Com

ViZDoomUnexpectedExitException: Controlled ViZDoom instance exited unexpectedly.